In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Revenue Forecast Model Comparisons

## Data

### Loading the data

In [2]:
#import raw revenue data
df_2016 = pd.read_csv("../../raw_data/orders2016.csv", sep=";")
df_2017 = pd.read_csv("../../raw_data/orders2017.csv", sep=";")
df_2018 = pd.read_csv("../../raw_data/orders2018.csv", sep=";")
df_2019 = pd.read_csv("../../raw_data/orders2019.csv", sep=";")
df_2020 = pd.read_csv("../../raw_data/orders2020.csv", sep=";")
df_2021 = pd.read_csv("../../raw_data/orders2021.csv", sep=";")
df_2022 = pd.read_csv("../../raw_data/orders2022.csv", sep=";")

df_list = [df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022]

#Dropping unnecessary columns, grouping by "date", summing "item_price" to get daily revenues

for i, df in enumerate(df_list):
    df_list[i] = pd.DataFrame(df.groupby(by="date")["item_price"].sum()/100)

#Concat all data in one dataframe, rename the columns for prophet

df = pd.concat(df_list, ignore_index=False)
df = df.rename(columns={"date": "ds", "item_price": "y"})
df["ds"] = df.index
df = df.reset_index(drop=True)
df = df[["ds","y"]]
df

#turning the ds (date) column into datetime

df['ds']=pd.to_datetime(df['ds'])

#Dropping outliers
df = df[df["y"]>=60]
df = df[df["y"]<=2300]
df = df.reset_index(drop=True)

In [5]:
#Loading additional regressor
feature_df = pd.read_csv("../../feature_data/final_feature_selection.csv")
feature_df["ds"] = pd.to_datetime(feature_df['ds'])

In [7]:
#Loading weather prediction data
weather_forecast = pd.read_csv("../../feature_data/finall_pred_weather.csv")
weather_forecast["ds"] = pd.to_datetime(weather_forecast["ds"])
weather_forecast["forecast dt iso"] = pd.to_datetime(weather_forecast["forecast dt iso"])

In [8]:
merged_df = pd.merge(df,feature_df,how="left")

In [9]:
merged_df.head()

,ds,y,temp,humidity,wind_speed,wind_deg,rain,clouds,Holiday,inflation_rate,Consumption Climate,cov_lock,unemp_Berlin_Mitte
0,2016-09-01,365.9,25.90,35,6.70,270,0.00,0,1,0.5,10.0,0,0.095
1,2016-09-02,358.6,22.18,46,4.63,280,0.00,75,1,0.5,10.0,0,0.095
2,2016-09-03,487.5,24.94,41,6.70,270,0.00,40,0,0.5,10.0,0,0.095
3,2016-09-04,90.9,19.94,73,6.70,230,0.00,75,0,0.5,10.0,0,0.095
4,2016-09-05,125.0,19.94,73,4.63,320,0.21,40,0,0.5,10.0,0,0.095


### Preparing data for different models

In [ ]:
#possible models: TimeSeries (Sarima, Arima, DNN), 

#1. introducing seasonal and weekly effects --> turning date into weekday information
#2. giving additional historical data --> revenue 1y ago, revenue past 1,2,3,7,14,30 days

## Splitting the data

In [87]:
#Setting variables
horizon = 16

#Splitting the data
split_date = "2022-10-14"
index_split = df[df["ds"]==split_date].index[0]
df_train = merged_df.iloc[:index_split]
df_test = merged_df.iloc[index_split:]
y_test = pd.DataFrame(df_test["y"])
weather_index_split = weather_forecast[weather_forecast["forecast dt iso"]==split_date].index[0]
weather_predict = weather_forecast.iloc[weather_index_split:weather_index_split+horizon,:]
weather_predict = weather_predict.drop(columns="forecast dt iso")

In [88]:
from prophet import Prophet
m = Prophet()
m.fit(df_train)

#Creating future dataframe
future = m.make_future_dataframe(periods=horizon)

#Adding feature values to future dataframe
future = pd.merge(future,feature_df,how="left")
future

#Update Future Timeframe with prediction weather data instead of historical weather data to prevent overfitting
cols_to_update = ['temp', 'humidity', 'clouds', 'wind_speed', 'wind_deg', 'rain']
future.loc[future.index[-(horizon):], cols_to_update] = weather_predict[cols_to_update].values

future

14:34:57 - cmdstanpy - INFO - Chain [1] start processing
14:34:58 - cmdstanpy - INFO - Chain [1] done processing


,ds,y,temp,humidity,wind_speed,wind_deg,rain,clouds,Holiday,inflation_rate,Consumption Climate,cov_lock,unemp_Berlin_Mitte
0,2016-09-01,365.9,25.90,35.0,6.70,270.0,0.00,0.0,1,0.5,10.0,0,0.095
1,2016-09-02,358.6,22.18,46.0,4.63,280.0,0.00,75.0,1,0.5,10.0,0,0.095
2,2016-09-03,487.5,24.94,41.0,6.70,270.0,0.00,40.0,0,0.5,10.0,0,0.095
3,2016-09-04,90.9,19.94,73.0,6.70,230.0,0.00,75.0,0,0.5,10.0,0,0.095
4,2016-09-05,125.0,19.94,73.0,4.63,320.0,0.21,40.0,0,0.5,10.0,0,0.095
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1676,2022-10-25,292.7,11.84,72.0,4.25,255.0,0.02,91.0,1,8.4,-41.9,0,0.090
1677,2022-10-26,346.6,10.73,66.0,3.66,279.0,0.00,99.0,1,8.4,-41.9,0,0.090
1678,2022-10-27,414.5,8.41,82.0,2.14,148.0,5.92,95.0,1,8.4,-41.9,0,0.090
1679,2022-10-28,620.0,9.00,68.0,2.50,275.0,0.00,61.0,1,8.4,-41.9,0,0.090


## Model

### XGBoost

In [89]:
from xgboost import XGBRegressor

X_train = df_train.drop(columns=["ds","y"])
y_train = df_train["y"]
X_test = future.tail(horizon).drop(columns=["ds","y"])
y_test = future.tail(horizon)["y"]

### Fitting

In [90]:
xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)
xgb.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

### Predicting

In [91]:
xgb_prediction = pd.DataFrame(xgb.predict(X_test),columns=["y_pred"])

y_test = pd.DataFrame(y_test).reset_index(drop=True)

### Evaluating

In [92]:
xgb_results = xgb_prediction
xgb_results["y_true"] = y_test["y"]
xgb_results["error"] = xgb_results["y_pred"] - xgb_results["y_true"]
xgb_results["mae"] = abs(xgb_results["error"])
xgb_results["mape"] = round(abs((xgb_results["y_pred"]/xgb_results["y_true"])-1)*100,ndigits=2)
xgb_results

,y_pred,y_true,error,mae,mape
0,257.959686,286.5,-28.540314,28.540314,9.96
1,334.308685,871.7,-537.391315,537.391315,61.65
2,283.512817,140.9,142.612817,142.612817,101.22
3,449.369476,632.8,-183.430524,183.430524,28.99
4,235.445068,226.9,8.545068,8.545068,3.77
5,289.328674,217.7,71.628674,71.628674,32.90
6,326.552826,260.2,66.352826,66.352826,25.50
7,364.067352,323.5,40.567352,40.567352,12.54
8,266.091919,743.1,-477.008081,477.008081,64.19
9,315.214355,379.4,-64.185645,64.185645,16.92


In [75]:
mae = xgb_results["mae"].mean()
mape = xgb_results["mape"].mean()
mae

164.09330825805662

In [76]:
mape

26.190624999999997

### Evaluating multiple dates

In [77]:
#Setting variables
horizon = 16
prediction_dates = ["2022-03-30","2022-04-09","2022-04-21","2022-05-01","2022-05-13","2022-05-25","2022-06-06","2022-06-18","2022-06-30",\
                   "2022-07-11","2022-07-23","2022-08-01","2022-08-13","2022-08-25","2022-09-02","2022-09-21","2022-10-02",\
                    "2022-10-14","2022-10-23","2022-11-05"]

# later days not working due to missing values in future holiday dataframe "2022-09-18" , "2022-09-06", ,"2022-09-30"]
predictions_mae = []
predictions_mape = []

#Looping through the dates to split into train and test set and make predictions

for date in prediction_dates:
    split_date = date
    index_split = df[df["ds"]==split_date].index[0]
    df_train = merged_df.iloc[:index_split]
    df_test = merged_df.iloc[index_split:]
    y_test = pd.DataFrame(df_test["y"])
    weather_index_split = weather_forecast[weather_forecast["forecast dt iso"]==split_date].index[0]
    weather_predict = weather_forecast.iloc[weather_index_split:weather_index_split+horizon,:]
    weather_predict = weather_predict.drop(columns="forecast dt iso")
    
    #Prophet Model for future dataframe
    m = Prophet()
    m = m.fit(df_train)
    
    #Creating future dataframe
    future = m.make_future_dataframe(periods=horizon)

    #Adding feature values to future dataframe
    future = pd.merge(future,feature_df,how="left")

    #Update Future Timeframe with prediction weather data instead of historical weather data to prevent overfitting
    cols_to_update = ['temp', 'humidity', 'clouds', 'wind_speed', 'wind_deg', 'rain']
    future.loc[future.index[-(horizon):], cols_to_update] = weather_predict[cols_to_update].values
    
    #Split data
    X_train = df_train.drop(columns=["ds","y"])
    y_train = df_train["y"]
    X_test = future.tail(horizon).drop(columns=["ds","y"])
    y_test = future.tail(horizon)["y"]

    #Instantiate
    
    xgb = XGBRegressor(max_depth=10, n_estimators=100, learning_rate=0.1)

    #Fitting the model
    
    xgb = xgb.fit(X_train, y_train)
    
    #Predicting
    
    xgb_prediction = pd.DataFrame(xgb.predict(X_test),columns=["y_pred"])
    y_test = pd.DataFrame(y_test).reset_index(drop=True)
    
    xgb_results = xgb_prediction
    xgb_results["y_true"] = y_test["y"]
    xgb_results["error"] = xgb_results["y_pred"] - xgb_results["y_true"]
    xgb_results["mae"] = abs(xgb_results["error"])
    xgb_results["mape"] = round(abs((xgb_results["y_pred"]/xgb_results["y_true"])-1)*100,ndigits=2)
    
    #Storing performance metrics
    predictions_mae.append(xgb_results["mae"].mean())
    predictions_mape.append(xgb_results["mape"].mean())

14:19:28 - cmdstanpy - INFO - Chain [1] start processing
14:19:28 - cmdstanpy - INFO - Chain [1] done processing
14:19:29 - cmdstanpy - INFO - Chain [1] start processing
14:19:29 - cmdstanpy - INFO - Chain [1] done processing
14:19:30 - cmdstanpy - INFO - Chain [1] start processing
14:19:30 - cmdstanpy - INFO - Chain [1] done processing
14:19:30 - cmdstanpy - INFO - Chain [1] start processing
14:19:31 - cmdstanpy - INFO - Chain [1] done processing
14:19:31 - cmdstanpy - INFO - Chain [1] start processing
14:19:32 - cmdstanpy - INFO - Chain [1] done processing
14:19:32 - cmdstanpy - INFO - Chain [1] start processing
14:19:32 - cmdstanpy - INFO - Chain [1] done processing
14:19:33 - cmdstanpy - INFO - Chain [1] start processing
14:19:33 - cmdstanpy - INFO - Chain [1] done processing
14:19:34 - cmdstanpy - INFO - Chain [1] start processing
14:19:34 - cmdstanpy - INFO - Chain [1] done processing
14:19:34 - cmdstanpy - INFO - Chain [1] start processing
14:19:34 - cmdstanpy - INFO - Chain [1]

In [83]:
xgb_mae = pd.DataFrame(predictions_mae).mean()
xgb_mape = pd.DataFrame(predictions_mape).mean()

In [86]:
predictions_mape

[53.566250000000004,
 49.675000000000004,
 28.991249999999997,
 44.764375,
 34.2775,
 50.97,
 34.40875,
 34.48875,
 69.48875000000001,
 14.705625,
 30.485625,
 26.190624999999997,
 44.20875,
 66.44749999999999,
 50.04625,
 61.956875000000004,
 39.751875,
 32.5825,
 32.818749999999994,
 127.78499999999998]

In [84]:
print(xgb_mae)
print(xgb_mape)

0    198.468297
dtype: float64
0    46.3805
dtype: float64
